# Hypothesis testing problems

## Exercise 1

You are a nutritionist researching two different types of diets to see if there is a significant difference in weight loss after one month. You choose two random groups of people; one group follows the first diet, and the other follows the second. At the end of the month, you record the weight loss (in kg) for each person:

| Diet 1 | Diet 2 |
|:-------|:-------|
| 2.0 | 3.0 |
| 2.5 | 3.2 |
| 3.0 | 3.1 |
| 2.8 | 2.9 |
| 2.3 | 2.8 |
| 2.7 | 3.0 |
| 2.5 | 3.2 |

With these data, it seeks to answer the following question: Is there a significant difference in average weight loss between people who followed the first diet and those who followed the second diet?

To draw conclusions, follow the points below:

- State the hypothesis: null and alternative hypothesis.
- Perform the test to test the hypothesis. You can use a Student's t-test.
- Analyze the conclusions.

**Hypothesis**
- H₀: μ₁ = μ₂ (no hay diferencia en la media de pérdida de peso).
- H₁: μ₁ ≠ μ₂ (sí hay diferencia).


In [5]:
import numpy as np
from scipy import stats

In [6]:
diet1 = np.array([2.0, 2.5, 3.0, 2.8, 2.3, 2.7, 2.5])
diet2 = np.array([3.0, 3.2, 3.1, 2.9, 2.8, 3.0, 3.2])


In [7]:
# 1. Assumtions: normality and homogeneity of variances

n1 = stats.shapiro(diet1)
n2 = stats.shapiro(diet2)
lev = stats.levene(diet1, diet2)

print("Normality of Diet1: W={:.3f}, p={:.3f}".format(n1.statistic, n1.pvalue))
print("Normality of Diet2: W={:.3f}, p={:.3f}".format(n2.statistic, n2.pvalue))
print("Homogeneity of variances: W={:.3f}, p={:.3f}".format(lev.statistic, lev.pvalue))

# 2. t-test (using equal_var according to homogeneity of variances)

t_stat, p_val = stats.ttest_ind(diet1, diet2, equal_var=(lev.pvalue > 0.05))
print("t-statistic: {:.3f}".format(t_stat))
print("p-valor: {:.4f}".format(p_val))


Normality of Diet1: W=0.981, p=0.964
Normality of Diet2: W=0.935, p=0.591
Homogeneity of variances: W=2.271, p=0.158
t-statistic: -3.538
p-valor: 0.0041


**3. Interpretation of p-value and effect measure**

*The probability of observing such a large difference between the means by pure chance is very low.*

Therefore, we reject the null hypothesis (H₀).

There is statistically significant evidence that the two diets result in different weight losses.

**Diet 2 produces a greater average weight loss than Diet 1.**

To understand whether the difference is relevant in practice, we calculate Cohen’s d:

A value around d ≈ 2 (approx.) indicates a very large effect.

This means that the difference is not only significant, but also very substantial.

Conclusions:

There is a clear and compelling difference between the two diets:

- Diet 2 consistently outperforms Diet 1 in weight loss.

- The result is statistically significant and has strong practical implications.

## Exercise 2

A farmer decides to test three different types of fertilizers to determine if one is superior in terms of corn production. The farmer plants corn on 15 identical plots and uses all three fertilizers (5 plots for each type). At the end of the season, he measures the corn yield (in kg) of each plot, with the following result:

| Fertilizer 1 | Fertilizer 2 | Fertilizer 3 |
|:-------------|:-------------|:-------------|
| 20 | 22 | 24 |
| 21 | 21 | 23 |
| 20 | 23 | 22 |
| 19 | 22 | 23 |
| 20 | 21 | 24 |

With this data, he seeks to answer the following question: Is there a significant difference in average corn yield between the three types of fertilizers?

To help you, follow the points below:

- State the hypothesis: null and alternative hypothesis.
- Perform the ANOVA test.
- Analyze the conclusions.
- If one fertilizer is better than another, how can we know it?

### Exercise 2 — Do the average yields differ among the three fertilizers?

**Hypothesis**
- H₀: μ₁ = μ₂ = μ₃ (the means are equal).

- H₁: At least one mean differs.

**Plan**
1. Check assumptions (normality within groups and homogeneity of variances).

2. Perform a one-way ANOVA.

3. If the ANOVA is significant → perform a Tukey post-hoc test to identify pairs that differ.

4. Calculate the effect size (eta-squared).

In [8]:
 # ANOVA + Tukey

import numpy as np
from scipy import stats
import pandas as pd
from statsmodels.stats.multicomp import pairwise_tukeyhsd

fert1 = [20, 21, 20, 19, 20]
fert2 = [22, 21, 23, 22, 21]
fert3 = [24, 23, 22, 23, 24]

# 1. Assumptions: normality by group and homogeneity of variances

print("Normalidad Fert1:", stats.shapiro(fert1))
print("Normalidad Fert2:", stats.shapiro(fert2))
print("Normalidad Fert3:", stats.shapiro(fert3))
print("Homogeneidad de varianza (3 grupos):", stats.levene(fert1, fert2, fert3))

# 2. ANOVA one-way

f_stat, p_val = stats.f_oneway(fert1, fert2, fert3)
print("ANOVA: F = {:.3f}, p = {:.4f}".format(f_stat, p_val))


# 3. Post-hoc Tukey (if ANOVA significant)

data = np.concatenate([fert1, fert2, fert3])
labels = ["F1"]*len(fert1) + ["F2"]*len(fert2) + ["F3"]*len(fert3)
tukey = pairwise_tukeyhsd(endog=data, groups=labels, alpha=0.05)
print("Resultados de Tukey HSD:")
print(tukey.summary())

# 4. Measure of effect (Eta-squared)

grand_mean = np.mean(data)  # Media total

# Between-group sum of squares

ss_between = (
    len(fert1) * (np.mean(fert1) - grand_mean)**2 +
    len(fert2) * (np.mean(fert2) - grand_mean)**2 +
    len(fert3) * (np.mean(fert3) - grand_mean)**2
)

# Total sum of squares

ss_total = np.sum((data - grand_mean)**2)

# Eta squared

eta_squared = ss_between / ss_total

print("Eta squared (η²):", eta_squared)


Normalidad Fert1: ShapiroResult(statistic=np.float64(0.8834906936019853), pvalue=np.float64(0.32542950716414576))
Normalidad Fert2: ShapiroResult(statistic=np.float64(0.8810376385817489), pvalue=np.float64(0.3140395561440371))
Normalidad Fert3: ShapiroResult(statistic=np.float64(0.8810376385817489), pvalue=np.float64(0.3140395561440371))
Homogeneidad de varianza (3 grupos): LeveneResult(statistic=np.float64(0.22222222222222207), pvalue=np.float64(0.8039599174006208))
ANOVA: F = 20.316, p = 0.0001
Resultados de Tukey HSD:
Multiple Comparison of Means - Tukey HSD, FWER=0.05
group1 group2 meandiff p-adj  lower  upper  reject
--------------------------------------------------
    F1     F2      1.8 0.0099 0.4572 3.1428   True
    F1     F3      3.2 0.0001 1.8572 4.5428   True
    F2     F3      1.4 0.0409 0.0572 2.7428   True
--------------------------------------------------
Eta squared (η²): 0.7719999999999995


**Results:**
- Shapiro-Wilk test by group: p > 0.05 → no violation of normality.

- Levene's test (3 groups): p > 0.05 → acceptable homogeneity of variances.

- ANOVA: F ≈ 20.316, p ≈ 0.0001 → p < 0.05, we reject H₀ (there are differences between means).

- Eta-squared ≈ 0.772 → very large effect (≈77% of the variability explained by the type of fertilizer).

**Tukey HSD (post-hoc):**
- F1 vs F2: mean difference = 1.8, p-adj ≈ 0.0099 → significant difference (F2 > F1). - F1 vs F3: mean difference = 3.2, p-adj ≈ 0.0001 → significant difference (F3 > F1).

- F2 vs F3: mean difference = 1.4, p-adj ≈ 0.0409 → significant difference (F3 > F2).

**Practical Conclusion:** Fertilizer 3 produces the highest average yield, followed by Fertilizer 2 and then Fertilizer 1; the differences between the three are statistically significant at α = 0.05.
